In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import cv2

2023-04-18 20:01:00.918669: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-18 20:01:01.079621: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-18 20:01:01.081203: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 20:01:01.890994: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def rgb2gray(rgb):
    return np.dot(rgb[..., :3], [3, 6, 1]).astype('int16')


def brighten(regImages):
    for ind in np.arange(len(regImages)):
        regImages[ind] = np.clip(regImages[ind] + int(np.random.normal(scale=400)), 0, 2550)
    return regImages


def trainValTest(length, trainPer, valPer):
    indices = np.arange(length)
    np.random.shuffle(indices)
    trainIndices = indices[0:int(trainPer*length)]
    valIndices = indices[int(trainPer*length):int((trainPer + valPer)*length)]
    testIndices = indices[int((trainPer + valPer)*length):]
    return trainIndices, valIndices, testIndices

In [3]:
path = r'data/asl_alphabet_train/asl_alphabet_train/'

types = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
         'W', 'X', 'Y', 'Z', 'space', 'del', 'nothing']

In [4]:
images = []
labels = []

for i in range(len(types)):
    file_list = os.listdir(path + types[i])
    for file in file_list:
        img = cv2.imread(path + types[i] + '/' + file)
        images.append(img)
        label = np.zeros(29)
        label[i] = 1
        labels.append(label)

images = np.array(images)
labels = np.array(labels)

In [5]:
trainInds, valInds, testInds = trainValTest(len(labels), 0.8, 0.1)

XTrain = images[trainInds]
XTrain = brighten(XTrain)
yTrain = labels[trainInds]

XVal = images[valInds]
yVal = labels[valInds]

XTest = images[testInds]
yTest = images[testInds]

In [6]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(3, 5, padding='same', activation='relu', input_shape=(200, 200, 1)),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(5, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(10, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(10, 5, strides=5, padding='valid', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(40, activation='relu'),
    tf.keras.layers.Dense(29, activation='softmax')])

net.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 3)       78        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 100, 100, 3)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 5)       140       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 50, 50, 5)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 50, 10)        460       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 25, 25, 10)       0

In [7]:
net.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
net.fit(XTrain, yTrain, epochs=30, batch_size=50, validation_data=(XVal, yVal))

2023-04-18 20:03:16.990050: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5736000000 exceeds 10% of free system memory.


Epoch 1/30


2023-04-18 20:03:51.012726: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 24000000 exceeds 10% of free system memory.
2023-04-18 20:03:51.208096: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 21850000 exceeds 10% of free system memory.
2023-04-18 20:03:51.208279: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 21850000 exceeds 10% of free system memory.
2023-04-18 20:03:51.214227: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 26100000 exceeds 10% of free system memory.


1434/1434 [==============================] - 294s 204ms/step - loss: 3.3270 - accuracy: 0.0773 - val_loss: 2.8629 - val_accuracy: 0.1713
Epoch 2/30
1434/1434 [==============================] - 281s 196ms/step - loss: 2.1086 - accuracy: 0.3450 - val_loss: 1.3632 - val_accuracy: 0.5321
Epoch 3/30
1434/1434 [==============================] - 286s 199ms/step - loss: 1.1412 - accuracy: 0.6039 - val_loss: 0.8544 - val_accuracy: 0.6946
Epoch 4/30
1434/1434 [==============================] - 290s 202ms/step - loss: 0.8152 - accuracy: 0.7108 - val_loss: 0.6273 - val_accuracy: 0.7748
Epoch 5/30
1434/1434 [==============================] - 295s 206ms/step - loss: 0.6380 - accuracy: 0.7709 - val_loss: 0.5256 - val_accuracy: 0.8187
Epoch 6/30
1434/1434 [==============================] - 292s 204ms/step - loss: 0.5360 - accuracy: 0.8081 - val_loss: 0.4734 - val_accuracy: 0.8332
Epoch 7/30
1434/1434 [==============================] - 292s 204ms/step - loss: 0.4707 - accuracy: 0.8313 - val_loss: 0.417

In [9]:
net.save("BenNN")

INFO:tensorflow:Assets written to: BenNN/assets


INFO:tensorflow:Assets written to: BenNN/assets


In [10]:
net.save_weights('BenNNweights')